In [ ]:
from awkward import Record as AwkwardRecord, Array as AwkwardArray
import awkward as ak

In [ ]:
FILENAME = "export-2024-11-12-16-59-20.parquet"
data = ak.from_parquet(FILENAME)[0]
display(data)

In [ ]:
from typing import Literal
import plotly.graph_objects as go
import plotly.express as px
import numpy as np


def to_scatter(
    key: str, dev_history: AwkwardArray, plot_key: Literal["value", "rssi"] = "value"
) -> go.Scatter:
    x = ak.to_numpy(dev_history["time"])
    if plot_key == "rssi":
        y = ak.to_numpy(dev_history["rssi"])
    elif plot_key == "value":
        y = ak.to_numpy(dev_history["value"])
    else:
        raise ValueError(f"Unknown plot_key: {plot_key}")
    return go.Scatter(x=x, y=y, mode="lines+markers", name=key)


scatters: list[go.Scatter] = []
for k in data.fields:
    val = data[k]
    scatters.append(to_scatter(k, val, "value"))

fig = go.Figure(scatters)

In [ ]:
display(fig)

In [ ]:
import csv
def dump_as_csv(mac:str, device_record: AwkwardRecord, filename:str):
    time = ak.to_numpy(device_record[mac]["time"]) # type: ignore
    value = ak.to_numpy(device_record[mac]["value"]) # type: ignore
    rssi = ak.to_numpy(device_record[mac]["rssi"]) # type: ignore
    with open(filename, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["time", "value", "rssi"])
        for t, v, r in zip(time, value, rssi):
            writer.writerow([t, v, r])

In [ ]:
dump_as_csv("A09E1AE4E710", data, "A09E1AE4E710.csv")